In [ ]:
!pip install svgwrite
!pip install cairosvg

In [ ]:
import requests
from PIL import Image
from io import BytesIO

# Replace 'YOUR_IMAGE_URL' with the URL of the image
image_url = 'https://images.unsplash.com/photo-1731943658442-804de2471a39?q=80&w=3648&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D'

# Download the image
response = requests.get(image_url)

if response.status_code == 200:
    # Open the downloaded image using PIL
    image = Image.open(BytesIO(response.content))

    # Set the minimum width you want to maintain
    min_width = 800

    # Calculate the new dimensions while maintaining the aspect ratio
    width, height = image.size
    if width < min_width:
        new_width = min_width
        new_height = int(height * (new_width / width))
    else:
        new_width = width
        new_height = height

    # Resize the image to the new dimensions
    resized_image = image.resize((new_width, new_height))

    # Save the resized image as PNG
    resized_image.save('image.png', 'PNG')

    # Display the downloaded image
    # display(Image(filename='image.jpg'))
else:
    print('Failed to download the image.')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import svgwrite
import random

def simplify_contour(contour, epsilon):
    # Apply Douglas-Peucker contour simplification
    return cv2.approxPolyDP(contour, epsilon, True)

# Load the original image
original_image = cv2.imread('image.png')

# Convert the image to the HSV color space
hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

# Define the increment for each color component
hue_increment = 25
saturation_increment = 25
value_increment = 25

color_ranges = []

for h in range(0, 180, hue_increment):
    for s in range(0, 256, saturation_increment):
        for v in range(0, 256, value_increment):
            # Define the lower and upper HSV range for the current color range
            lower_bound = np.array([h, s, v])
            upper_bound = np.array([h + hue_increment, s + saturation_increment, v + value_increment])
            color_ranges.append((lower_bound, upper_bound))

# Create an SVG drawing

svg_width = 500
svg_height = 300
# Create an SVG drawing with specified dimensions and viewBox
dwg = svgwrite.Drawing('color_regions.svg', profile='tiny', size=(new_width, new_height))

# Set the viewBox attribute
dwg.viewbox(0, 0, new_width, new_height)

# Process each color region and convert it to an SVG shape
for i, (lower_bound, upper_bound) in enumerate(color_ranges):
    # Create a mask based on the specified HSV range
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:

        epsilon = 0.01 * cv2.arcLength(contour, True)  # Adjust epsilon as needed
        simplified_contour = simplify_contour(contour, epsilon)

        # Create a path based on the simplified contour
        path_data = 'M '
        for point in simplified_contour:
            x, y = point[0]
            path_data += f'{x},{y} '
        path_data += 'Z'

        # Define a gradient with random colors
        gradient_id = 'gradient-' + str(random.randint(1, 1000))
        gradient = svgwrite.gradients.LinearGradient(start=(0, 0), end=(1, 0), id=gradient_id)
        for _ in range(2):  # Define two gradient stops
            gradient.add_stop_color(offset='0%', color='#{:02X}{:02X}{:02X}'.format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))
        dwg.defs.add(gradient)

        # Create an SVG path filled with the gradient
        path = svgwrite.path.Path(d=path_data, fill=f'url(#{gradient_id})')
        dwg.add(path)

# Save the SVG file
dwg.save()


import cairosvg
from IPython.display import Image

# Replace 'your_svg_file.svg' with the path to your SVG file
svg_file = 'color_regions.svg'
png_file = 'output1.png'  # Output PNG file

# Set the size explicitly (in this example, width is 500 and height is 300)
cairosvg.svg2png(url=svg_file, write_to=png_file)

# Display the PNG image
from IPython.display import Image, display

# Display the original image
display(Image(filename='image.png'))

# Display the PNG image
display(Image(filename=png_file))

In [ ]:
import cv2
import numpy as np
import svgwrite

# Load the original image
original_image = cv2.imread('image.png')

# Convert the image to the HSV color space
hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

# Define the increment for each color component
hue_increment = 45
saturation_increment = 45
value_increment = 45

color_ranges = []

for h in range(0, 180, hue_increment):
    for s in range(0, 256, saturation_increment):
        for v in range(0, 256, value_increment):
            # Define the lower and upper HSV range for the current color range
            lower_bound = np.array([h, s, v])
            upper_bound = np.array([h + hue_increment, s + saturation_increment, v + value_increment])
            color_ranges.append((lower_bound, upper_bound))

dwg = svgwrite.Drawing('color_regions.svg', profile='tiny', size=(new_width, new_height))

# Set the viewBox attribute
dwg.viewbox(0, 0, new_width, new_height)

# Process each color region and convert it to an SVG shape
for i, (lower_bound, upper_bound) in enumerate(color_ranges):
    # Create a mask based on the specified HSV range
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    combined_path = svgwrite.container.Group()
    for contour in contours:
        if len(contour) >= 6:  # Ignore small contours
            # Apply contour simplification with a less aggressive epsilon value
            epsilon = 0.001 * cv2.arcLength(contour, True)  # Adjust epsilon as needed
            simplified_contour = cv2.approxPolyDP(contour, epsilon, True)

            # Sample the color from the original image for the gradient
            color_sample = original_image[simplified_contour[0][0][1], simplified_contour[0][0][0]]

            # Create an SVG path for the simplified contour
            path_data = 'M '
            for point in simplified_contour:
                x, y = point[0]
                path_data += f'{x},{y} '
            path_data += 'Z'
            path = svgwrite.path.Path(d=path_data, fill=f'rgb({color_sample[2]},{color_sample[1]},{color_sample[0]})')
            combined_path.add(path)

    # Add the combined path to the drawing
    dwg.add(combined_path)

# Save the SVG file
dwg.save()



import cairosvg
from IPython.display import Image

# Replace 'your_svg_file.svg' with the path to your SVG file
svg_file = 'color_regions.svg'
png_file = 'output2.png'  # Output PNG file

# Set the size explicitly (in this example, width is 500 and height is 300)
cairosvg.svg2png(url=svg_file, write_to=png_file)

# Display the PNG image
from IPython.display import Image, display
import os

png_file_size = os.path.getsize(png_file)

# Get the file size for the SVG image
svg_file_size = os.path.getsize(svg_file)

print(f'PNG File Size: {png_file_size} bytes')
print(f'SVG File Size: {svg_file_size} bytes')

# Display the original image
display(Image(filename='image.png'))

# Display the PNG image
display(Image(filename=png_file))



In [ ]:
import cv2
import numpy as np
import svgwrite

# Load the original image
original_image = cv2.imread('image.png')

# Convert the image to the HSV color space
hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

# Define the increment for each color component
hue_increment = 45
saturation_increment = 45
value_increment = 45

color_ranges = []

for h in range(0, 180, hue_increment):
    for s in range(0, 256, saturation_increment):
        for v in range(0, 256, value_increment):
            # Define the lower and upper HSV range for the current color range
            lower_bound = np.array([h, s, v])
            upper_bound = np.array([h + hue_increment, s + saturation_increment, v + value_increment])
            color_ranges.append((lower_bound, upper_bound))

# Create an SVG drawing with specified dimensions and viewBox
dwg = svgwrite.Drawing('color_regions.svg', profile='tiny', size=(new_width, new_height))
dwg.viewbox(0, 0, new_width, new_height)

# Process each color region and convert it to an SVG shape
for i, (lower_bound, upper_bound) in enumerate(color_ranges):
    # Create a mask based on the specified HSV range
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if len(contour) >= 6:  # Ignore small contours
            # Apply contour simplification with a less aggressive epsilon value
            epsilon = 0.005 * cv2.arcLength(contour, True)  # Adjust epsilon as needed
            simplified_contour = cv2.approxPolyDP(contour, epsilon, True)

            # Sample the color from the original image for the gradient
            color_sample = original_image[simplified_contour[0][0][1], simplified_contour[0][0][0]]

            # Define a gradient with the sampled color
            gradient_id = 'gradient-' + str(i)
            gradient = svgwrite.gradients.LinearGradient(start=(0, 0), end=(1, 0), id=gradient_id)
            gradient.add_stop_color(offset='0%', color='rgb({},{},{})'.format(color_sample[2], color_sample[1], color_sample[0]))
            gradient.add_stop_color(offset='100%', color='rgb({},{},{})'.format(color_sample[2], color_sample[1], color_sample[0]))
            dwg.defs.add(gradient)

            # Create an SVG path filled with the gradient and set border color to match
            path_data = 'M '
            for point in simplified_contour:
                x, y = point[0]
                path_data += f'{x},{y} '
            path_data += 'Z'
            path = svgwrite.path.Path(d=path_data, fill=f'url(#{gradient_id})', stroke=f'rgb({color_sample[2]},{color_sample[1]},{color_sample[0]})')
            dwg.add(path)

# Save the SVG file
dwg.save()


# Display the PNG image
from IPython.display import Image, display
import os

png_file_size = os.path.getsize(png_file)

# Get the file size for the SVG image
svg_file_size = os.path.getsize(svg_file)

print(f'PNG File Size: {png_file_size} bytes')
print(f'SVG File Size: {svg_file_size} bytes')

# Display the original image
display(Image(filename='image.png'))

# Display the PNG image
display(Image(filename=png_file))

In [ ]:
import cv2
import numpy as np
import svgwrite

# Load the original image in grayscale
original_image = cv2.imread('image.png', cv2.IMREAD_GRAYSCALE)

new_width, new_height = original_image.shape[1], original_image.shape[0]

# Define a threshold value for binarization
threshold_value = 100

# Threshold the grayscale image
_, thresholded_image = cv2.threshold(original_image, threshold_value, 255, cv2.THRESH_BINARY)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

dwg = svgwrite.Drawing('grayscale_regions.svg', profile='tiny', size=(new_width, new_height))

dwg.viewbox(0, 0, new_width, new_height)

combined_path = svgwrite.container.Group()
for contour in contours:
    if len(contour) >= 6:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        simplified_contour = cv2.approxPolyDP(contour, epsilon, True)
        path_data = 'M '
        for point in simplified_contour:
            x, y = point[0]
            path_data += f'{x},{y} '
        path_data += 'Z'
        path = svgwrite.path.Path(d=path_data, fill='black')
        combined_path.add(path)

dwg.add(combined_path)

dwg.save()

import cairosvg
from IPython.display import Image

svg_file = 'grayscale_regions.svg'
png_file = 'output.png'

cairosvg.svg2png(url=svg_file, write_to=png_file)

from IPython.display import Image, display
import os

png_file_size = os.path.getsize(png_file)
svg_file_size = os.path.getsize(svg_file)

display(Image(filename='image.png'))
display(Image(filename=png_file))



In [ ]:
import cv2
import numpy as np
import svgwrite

# Load the original image in grayscale
original_image = cv2.imread('image.png', cv2.IMREAD_GRAYSCALE)

new_width, new_height = original_image.shape[1], original_image.shape[0]

# Create an empty SVG
dwg = svgwrite.Drawing('grayscale_regions.svg', profile='tiny', size=(new_width, new_height))

dwg.viewbox(0, 0, new_width, new_height)

# Define a range for threshold values for grayscale effect
min_threshold = 0
max_threshold = 255

# Define the number of steps for the grayscale effect
num_steps = 20

# Calculate the step size for thresholding
step_size = (max_threshold - min_threshold) / num_steps

combined_path = svgwrite.container.Group()
for step in range(num_steps):
    # Calculate the threshold value for this step
    threshold_value = int(min_threshold + step * step_size)

    # Threshold the grayscale image
    _, thresholded_image = cv2.threshold(original_image, threshold_value, 255, cv2.THRESH_BINARY)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if len(contour) >= 6:
            epsilon = 0.01 * cv2.arcLength(contour, True)
            simplified_contour = cv2.approxPolyDP(contour, epsilon, True)
            path_data = 'M '
            for point in simplified_contour:
                x, y = point[0]
                path_data += f'{x},{y} '
            path_data += 'Z'
            # Use step-dependent grayscale fill
            fill_value = 255 - step * int(255 / num_steps)
            path = svgwrite.path.Path(d=path_data, fill=f'rgb({fill_value},{fill_value},{fill_value})')
            combined_path.add(path)

dwg.add(combined_path)

dwg.save()

import cairosvg
from IPython.display import Image

svg_file = 'grayscale_regions.svg'
png_file = 'output.png'

cairosvg.svg2png(url=svg_file, write_to=png_file)

from IPython.display import Image, display
import os

png_file_size = os.path.getsize(png_file)
svg_file_size = os.path.getsize(svg_file)

display(Image(filename='image.png'))
display(Image(filename=png_file))


In [ ]:
import cv2
import numpy as np
import svgwrite

# Load the original image in grayscale
original_image = cv2.imread('image.png', cv2.IMREAD_GRAYSCALE)

new_width, new_height = original_image.shape[1], original_image.shape[0]

# Create an empty SVG
dwg = svgwrite.Drawing('bw_gradient_regions.svg', profile='tiny', size=(new_width, new_height))

dwg.viewbox(0, 0, new_width, new_height)

# Define the number of steps for the gradient effect
num_steps = 10

# Calculate step size for thresholding
step_size = 256 / num_steps

combined_path = svgwrite.container.Group()
for step in range(num_steps):
    # Calculate the upper threshold for this step
    upper_threshold = int((step + 1) * step_size)

    # Apply binary thresholding
    _, thresholded = cv2.threshold(original_image, upper_threshold, 255, cv2.THRESH_BINARY)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if len(contour) >= 6:
            epsilon = 0.01 * cv2.arcLength(contour, True)
            simplified_contour = cv2.approxPolyDP(contour, epsilon, True)
            path_data = 'M '
            for point in simplified_contour:
                x, y = point[0]
                path_data += f'{x},{y} '
            path_data += 'Z'
            # Use step-dependent grayscale fill
            fill_value = 0 if step % 2 == 0 else 255
            path = svgwrite.path.Path(d=path_data, fill=f'rgb({fill_value},{fill_value},{fill_value})')
            combined_path.add(path)

dwg.add(combined_path)

dwg.save()

import cairosvg
from IPython.display import Image

svg_file = 'bw_gradient_regions.svg'
png_file = 'output.png'

cairosvg.svg2png(url=svg_file, write_to=png_file)

from IPython.display import Image, display
import os

png_file_size = os.path.getsize(png_file)
svg_file_size = os.path.getsize(svg_file)

display(Image(filename='image.png'))
display(Image(filename=png_file))



In [ ]:
from PIL import Image, ImageOps
from IPython.display import display

# Open the image
image = Image.open('image.png')

# Convert it to grayscale
gray_image = ImageOps.grayscale(image)

# Dither the grayscale image
bw_image = gray_image.convert('1', dither=Image.FLOYDSTEINBERG)

# Save the black and white dithered image
bw_image.save('bw_dithered_image.png')

# Display the original and black and white dithered images
display(image)
display(bw_image)


In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/800x800')
image = Image.open(BytesIO(response.content))

gray_image = ImageOps.grayscale(image)
bw_image = gray_image.convert('1', dither=Image.FLOYDSTEINBERG)
bw_image.save('bw_dithered_image.png')

display(image)
display(bw_image)


In [ ]:
from PIL import Image
from IPython.display import display
import requests
from io import BytesIO

def create_three_color_image(image, red_intensity=128, brightness=200):
    bw_image = image.convert('L')

    # Create an empty result image with a custom palette (black, white, red)
    result_image = Image.new('P', bw_image.size)

    # Define the palette with three colors (black, white, red)
    palette = [0, 0, 0, 255, 255, 255, red_intensity, 0, 0]

    # Set the palette for the result image
    result_image.putpalette(palette)

    # Convert grayscale values to palette indices
    bw_image = bw_image.point(lambda p: 0 if p < red_intensity else (1 if p < brightness else 2))

    # Paste the dithered image into the result image
    result_image.paste(bw_image, (0, 0))

    return result_image

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/800x800')
image = Image.open(BytesIO(response.content))

# Create the three-color image (black, white, and red) and save it as a PNG
brightness_value = 160  # Adjust this value for the desired brightness
three_color_image = create_three_color_image(image, red_intensity=128, brightness=brightness_value)
three_color_image.save('three_color_image.png', format='PNG')

display(image)
display(three_color_image)


In [ ]:
from PIL import Image, ImageOps, ImageFilter
from IPython.display import display
import requests
from io import BytesIO

def create_three_color_image(image, red_intensity=128, brightness=128):
    # Convert the image to grayscale
    gray_image = ImageOps.grayscale(image)

    # Create an empty result image with a custom palette (black, white, red)
    result_image = Image.new('P', gray_image.size)

    # Define the palette with three colors (black, white, red)
    palette = [0, 0, 0, 255, 255, 255, red_intensity, 0, 0]

    # Set the palette for the result image
    result_image.putpalette(palette)

    # Convert grayscale values to palette indices
    bw_image = gray_image.point(lambda p: 0 if p < red_intensity else (1 if p < brightness else 2))

    # Paste the dithered image into the result image
    result_image.paste(bw_image, (0, 0))

    return result_image

# Fetch the image from the URL
#response = requests.get('https://source.unsplash.com/random/800x800')
#image = Image.open(BytesIO(response.content))

# Create the three-color image (black, white, and red) and save it as a PNG
brightness_value = 200  # Adjust this value for the desired brightness
three_color_image = create_three_color_image(image, red_intensity=128, brightness=brightness_value)

# Apply dithering to the three-color image (Floyd-Steinberg dithering)
dithered_image = three_color_image.convert('RGB').convert('P', dither=Image.FLOYDSTEINBERG)

display(image)
display(three_color_image)
display(dithered_image)


In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/800x800')
image = Image.open(BytesIO(response.content))

# Display the original color image
display(image)

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Display the grayscale version
display(gray_image)

# Create an empty result image
result_image = Image.new('RGB', image.size)

# Iterate through the pixels and determine the color (black, red, or white)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))
        if pixel < 128:
            result_image.putpixel((x, y), (0, 0, 0))  # Black
        elif pixel < 192:
            result_image.putpixel((x, y), (255, 0, 0))  # Red
        else:
            result_image.putpixel((x, y), (255, 255, 255))  # White

# Display the result image with the black, red, and white areas
display(result_image)



In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/800x800')
image = Image.open(BytesIO(response.content))

# Display the original color image
display(image)

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Display the grayscale version
display(gray_image)

# Create an empty result image
result_image = Image.new('RGB', image.size)

# Define the number of segments for black, red, and white
num_segments = 3

# Iterate through the pixels and determine the color (black, red, or white)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        if pixel < segment_size:
            result_image.putpixel((x, y), (0, 0, 0))  # Black
        elif pixel < 2 * segment_size:
            # Calculate the red intensity within the segment
            red_intensity = (pixel - segment_size) * (255 // segment_size)
            result_image.putpixel((x, y), (red_intensity, 0, 0))  # Red
        else:
            # Calculate the white intensity within the segment
            white_intensity = (pixel - 2 * segment_size) * (255 // segment_size)
            result_image.putpixel((x, y), (255, white_intensity, white_intensity))  # White

# Display the result image with black, red, and white areas
display(result_image)



In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/800x800')
image = Image.open(BytesIO(response.content))

# Display the original color image
display(image)

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Display the grayscale version
display(gray_image)

# Create an empty result image
result_image = Image.new('RGB', image.size)

# Define the number of segments for black, red, and white
num_segments = 3

# Iterate through the pixels and determine the color (black, less red, or white)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        if pixel < segment_size:
            result_image.putpixel((x, y), (0, 0, 0))  # Black
        elif pixel < 2 * segment_size:
            # Calculate the red intensity within the segment
            red_intensity = (pixel - segment_size) * (255 // segment_size)
            result_image.putpixel((x, y), (red_intensity, 0, 0))  # Red
        else:
            # Calculate the white intensity within the segment
            white_intensity = (pixel - 2 * segment_size) * (255 // segment_size)
            result_image.putpixel((x, y), (255, white_intensity, white_intensity))  # White

# Display the result image with black, less red, and white areas
display(result_image)

# Apply Floyd-Steinberg dithering to the result image
dithered_image = result_image.convert('RGB').convert('P', dither=Image.FLOYDSTEINBERG)

# Display the dithered image
display(dithered_image)



In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/200x200')
image = Image.open(BytesIO(response.content))

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Create an empty result image
result_image = Image.new('RGB', image.size)

# Define the number of segments for black, red, and white
num_segments = 3

# Define the colors
colors = [(0, 0, 0), (255, 0, 0), (255, 255, 255)]  # Black, Red, White

# Iterate through the pixels and assign one of the three colors
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        # Determine which color to assign
        segment_index = min(pixel // segment_size, num_segments - 1)
        color = colors[segment_index]

        result_image.putpixel((x, y), color)

combined_image = Image.new('RGB', (image.width * 2, image.height))
combined_image.paste(image, (0, 0))
combined_image.paste(result_image, (image.width, 0))

# Display the combined image
display(combined_image)



In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/800x800')
image = Image.open(BytesIO(response.content))

# Display the original color image
display(image)

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Create empty result images for combined gradients
combined_image1 = Image.new('RGB', image.size)
combined_image2 = Image.new('RGB', image.size)

# Define the number of segments for black, red, and white
num_segments = 3

# Iterate through the pixels and determine the color (black, red, or white gradients)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        if pixel < segment_size:
            # Calculate the red intensity within the segment
            red_intensity = pixel * (255 // segment_size)
            combined_image1.putpixel((x, y), (red_intensity, 0, 0))  # Red
            combined_image2.putpixel((x, y), (0, 0, 0))  # Black
        elif pixel < 2 * segment_size:
            combined_image1.putpixel((x, y), (255, 255, 255))  # White
            combined_image2.putpixel((x, y), (red_intensity, 0, 0))  # Red
        else:
            # Calculate the white intensity within the segment
            white_intensity = (pixel - 2 * segment_size) * (255 // segment_size)
            combined_image1.putpixel((x, y), (255, white_intensity, white_intensity))  # White
            combined_image2.putpixel((x, y), (255, white_intensity, white_intensity))  # White

# Display the first combined gradient image
display(combined_image1)

# Display the second combined gradient image
display(combined_image2)



In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/800x800')
image = Image.open(BytesIO(response.content))

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Create empty result images for black-to-white and red-to-white gradients
black_white_image = Image.new('RGB', image.size)
red_white_image = Image.new('RGB', image.size)

# Define the number of segments for black, red, and white
num_segments = 3

# Iterate through the pixels and determine the color (black, red, or white gradients)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        if pixel < segment_size:
            black_intensity = (segment_size - pixel) * (255 // segment_size)
            black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))  # Black-to-White
            red_white_image.putpixel((x, y), (255, 255, 255))  # White
        elif pixel < 2 * segment_size:
            # Calculate the red intensity within the segment
            red_intensity = (pixel - segment_size) * (255 // segment_size)
            black_white_image.putpixel((x, y), (255, red_intensity, red_intensity))  # Red
            red_white_image.putpixel((x, y), (255, 255, 255))  # White
        else:
            # Calculate the white intensity within the segment
            white_intensity = (pixel - 2 * segment_size) * (255 // segment_size)
            black_white_image.putpixel((x, y), (255, white_intensity, white_intensity))  # White
            red_white_image.putpixel((x, y), (255, white_intensity, white_intensity))  # White

# Combine the original image, black-to-white, and red-to-white gradient images side by side
combined_image = Image.new('RGB', (image.width * 3, image.height))
combined_image.paste(image, (0, 0))
combined_image.paste(black_white_image, (image.width, 0))
combined_image.paste(red_white_image, (image.width * 2, 0))

# Display the combined image
display(combined_image)


In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/800x800')
image = Image.open(BytesIO(response.content))

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Create an empty result image for the combined gradients
combined_image = Image.new('RGB', (image.width * 3, image.height))

# Define the number of segments for black, red, and white
num_segments = 3

# Create empty images for black/white and red gradients
black_white_image = Image.new('RGB', image.size)
red_image = Image.new('RGB', image.size)

# Iterate through the pixels and determine the color (black/white or red gradients)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        if pixel < segment_size:
            black_intensity = (segment_size - pixel) * (255 // segment_size)
            black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))  # Black-to-White
        elif pixel < 2 * segment_size:
            red_intensity = (pixel - segment_size) * (255 // segment_size)
            red_image.putpixel((x, y), (255, red_intensity, red_intensity))  # Red

# Combine the original image, black/white, and red gradients side by side
combined_image.paste(image, (0, 0))
combined_image.paste(black_white_image, (image.width, 0))
combined_image.paste(red_image, (image.width * 2, 0))

# Display the combined image
display(combined_image)


In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/2000x2000')
image = Image.open(BytesIO(response.content))

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Create an empty result image for the combined gradients
combined_image = Image.new('RGB', (image.width * 3, image.height))
combined2_image = Image.new('RGB', (image.width * 3, image.height))
combined3_image = Image.new('RGB', (image.width * 2, image.height))
# Define the number of segments for black, red, and white
num_segments = 3

# Create empty images for black/white and red gradients
black_white_image = Image.new('RGB', image.size)
red_image = Image.new('RGB', image.size)

# Iterate through the pixels and determine the color (black/white or red gradients)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        if pixel < segment_size:
            black_intensity = (segment_size - pixel) * (255 // segment_size)
            black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))  # Black-to-White
        elif pixel < 2 * segment_size:
            red_intensity = (pixel - segment_size) * (255 // segment_size)
            red_image.putpixel((x, y), (255, red_intensity, red_intensity))  # Red

# Apply Floyd-Steinberg dithering to reduce black/white and red gradients to 2 colors
black_white_dithered = black_white_image.convert('1', dither=Image.FLOYDSTEINBERG)
red_dithered = red_image.convert('1', dither=Image.FLOYDSTEINBERG)
red2_dithered = ImageOps.invert(red_dithered)
red_dithered = ImageOps.invert(red_dithered)
black_white_dithered = ImageOps.invert(black_white_dithered)

red = (255, 0, 0)

# Create a new image with a transparent background
transparent_image = Image.new('RGBA', red_dithered.size, (0, 0, 0, 0))

# Load pixel data
image_data = red_dithered.load()
transparent_data = transparent_image.load()
width, height = red_dithered.size

for x in range(width):
    for y in range(height):
        if image_data[x, y] == 0:  # Check for black pixels
            transparent_data[x, y] = red


transparent_image.save('output_image.png')

# Combine the original image, dithered black/white, and dithered red gradients side by side
combined_image.paste(image, (0, 0))
combined_image.paste(black_white_dithered, (image.width, 0))
combined_image.paste(transparent_image, (image.width * 2, 0))

display(combined_image)

# Create a combined image with the same dimensions as black_white_dithered
combined_image = Image.new('RGBA', black_white_dithered.size)

# Paste black_white_dithered as the background
combined_image.paste(black_white_dithered, (0, 0))
# Paste transparent_image on top of black_white_dithered
combined_image.paste(transparent_image, (0, 0), mask=transparent_image)
# Save the combined image
combined_image.save('combined_image.png')


combined2_image.paste(combined_image, (0, 0))
combined2_image.paste(black_white_dithered, (image.width, 0))
combined2_image.paste(red2_dithered, (image.width * 2, 0))


combined3_image.paste(image, (0, 0))
combined3_image.paste(combined_image, (image.width, 0))

combined3_image.save('comparison.png')
combined3_image.save('comparison.pdf', 'PDF')
# Display the combined image
display(combined2_image)


In [ ]:
from PIL import Image, ImageOps, ImageDraw
from IPython.display import display
import requests
from io import BytesIO
import zlib
import math
import argparse


# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/200x200')
image = Image.open(BytesIO(response.content))

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Create an empty result image for the combined gradients


# Define the number of segments for black, custom color, and white
num_segments = 3

# Create empty images for black/white and custom color gradients
black_white_image = Image.new('RGB', image.size)
custom_color_image = Image.new('RGB', image.size)

# Define your custom color (e.g., green as (0, 255, 0))
custom_color = (255, 0, 0)

# Iterate through the pixels and determine the color (black/white or custom color gradients)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        if pixel < segment_size:
            black_intensity = (segment_size - pixel) * (255 // segment_size)
            black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))  # Black-to-White
        elif pixel < 2 * segment_size:
            # Calculate the intensity of the custom color based on the pixel value
            custom_intensity = (pixel - segment_size) * (255 // segment_size)
            custom_color_image.putpixel((x, y), custom_color)  # Custom Color

# Apply Floyd-Steinberg dithering to reduce black/white and custom color gradients to 2 colors
black_white_dithered = black_white_image.convert('1', dither=Image.FLOYDSTEINBERG)
custom_color_dithered = custom_color_image.convert('1', dither=Image.FLOYDSTEINBERG)

# Create a new image with a transparent background
transparent_image = Image.new('RGBA', custom_color_dithered.size, (0, 0, 0, 0))

# Load pixel data
custom_color_data = custom_color_dithered.load()
transparent_data = transparent_image.load()
width, height = custom_color_dithered.size


img = Image.new('RGBA', black_white_dithered.size)

for x in range(width):
    for y in range(height):
        if custom_color_data[x, y] == 0:  # Check for black pixels
            img.putpixel((x, y), (0, 0, 0, 0))
        else:
            img.putpixel((x, y), custom_color + (255,))




# Save the image as a PNG file with transparency

combined_output = Image.new('RGB', (image.width, image.height))
combined_output.paste(black_white_dithered, (0, 0))
combined_output.paste(img, (0, 0), mask=img)


combined_image = Image.new('RGB', (image.width * 2, image.height))
# Combine the original image, dithered black/white, and dithered custom color gradients side by side


combined_image.paste(image, (0, 0))
combined_image.paste(combined_output, (image.width, 0))


display(combined_image)
combined_image.save('combined image.pdf', 'PDF')

img.save('image.png')

# Save the image as JPEG
img.save('image.jpg', quality=90)  # Adjust quality as needed



In [ ]:
from PIL import Image, ImageOps, ImageDraw
from IPython.display import display
import requests
from io import BytesIO
import zlib
import math
import argparse


# Fetch the image from the URL
response = requests.get('https://source.unsplash.com/random/200x200')
image = Image.open(BytesIO(response.content))

# Convert the image to grayscale
gray_image = ImageOps.grayscale(image)

# Create an empty result image for the combined gradients


# Define the number of segments for black, custom color, and white
num_segments = 3

# Create empty images for black/white and custom color gradients
black_white_image = Image.new('RGB', image.size)
custom_color_image = Image.new('RGB', image.size)

# Define your custom color (e.g., green as (0, 255, 0))
custom_color = (255, 0, 0)

# Iterate through the pixels and determine the color (black/white or custom color gradients)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))

        # Calculate the segment size
        segment_size = 256 // num_segments

        if pixel < segment_size:
            black_intensity = (segment_size - pixel) * (255 // segment_size)
            black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))  # Black-to-White
        elif pixel < 2 * segment_size:
            # Calculate the intensity of the custom color based on the pixel value
            custom_intensity = (pixel - segment_size) * (255 // segment_size)
            custom_color_image.putpixel((x, y), custom_color)  # Custom Color

# Apply Floyd-Steinberg dithering to reduce black/white and custom color gradients to 2 colors
black_white_dithered = black_white_image.convert('1', dither=Image.FLOYDSTEINBERG)
custom_color_dithered = custom_color_image.convert('1', dither=Image.FLOYDSTEINBERG)

display(image)
#display(black_white_dithered)
#display(custom_color_dithered)


custom_color_data = custom_color_dithered.load()
img = Image.new('RGBA', black_white_dithered.size)




for x in range(width):
    for y in range(height):
        if custom_color_data[x, y] == 0:  # Check for black pixels
            img.putpixel((x, y), (0, 0, 0, 0))
        else:
            img.putpixel((x, y), custom_color + (255,))


black_white_data = black_white_dithered.load()
img_data = img.load()



black_white_rgb = black_white_dithered.convert('RGB')



# Iterate through the pixels and replace red pixels in 'black_white_dithered' with corresponding pixels from 'img'
for x in range(black_white_dithered.width):
    for y in range(black_white_dithered.height):
        if img_data[x, y] == custom_color + (255,):  # Check for red pixels in 'img' (assuming 1 represents red in 'img')
            black_white_rgb.putpixel((x, y), custom_color + (255,))  # Replace corresponding pixel in 'black_white_dithered' with 'img'


# Display the updated image
display(black_white_rgb)

black_white_rgb.save('image.png')

# Save the image as JPEG
black_white_rgb.save('image.jpg')  # Adjust quality as needed


image_path = 'image.png'  # Replace this with the path to your image
image = Image.open(image_path)
image = image.convert('L')

# Resize the image to match the dimensions in your provided array
width = 25  # Replace this with the width of your image as specified in the comments
height = 200  # Replace this with the height of your image as specified in the comments
image = image.resize((width, height))

# Get pixel data as a list of values
pixel_values = list(image.getdata())

# Convert pixel values to hexadecimal representation similar to the given array
hex_values = [f'0x{val:02x}' for val in pixel_values]

# Group hexadecimal values into lines to match the format in the array (25 bytes per line)
lines = [hex_values[i:i + width] for i in range(0, len(hex_values), width)]

# Convert lines to strings and concatenate into a single string with commas and line breaks
output_lines = [', '.join(line) for line in lines]
output = ',\n'.join(output_lines)

# Write the formatted hexadecimal values to a file with the specified header
output_file_path = 'output_image_data.h'  # Replace this with your desired output file path
header = '''#ifndef PROGMEM
#define PROGMEM
#endif

'''

with open(output_file_path, 'w') as output_file:
    output_file.write(header)
    output_file.write('const uint8_t image_0[] PROGMEM = {\n')
    output_file.write(output)
    output_file.write('\n};\n')



In [ ]:
from PIL import Image, ImageOps, ImageDraw
from IPython.display import display
import requests
from io import BytesIO
import zlib
import math
import argparse
from google.colab import files

def generate_new_image(i):
  global black_white_dithered, custom_color_dithered
  # Fetch the image from the URL
  response = requests.get('https://source.unsplash.com/random/50x50')
  image = Image.open(BytesIO(response.content))
  #image2 = Image.open(str(i) + ".jpg")
  #image = image2.resize((200, 200))
  display(image)
  # Convert the image to grayscale
  gray_image = ImageOps.grayscale(image)
  # Define the number of segments for black, custom color, and white
  num_segments = 3
  # Create empty images for black/white and custom color gradients
  black_white_image = Image.new('RGB', image.size)
  custom_color_image = Image.new('RGB', image.size)
  # Define your custom color (e.g., green as (0, 255, 0))
  custom_color = (255, 0, 0)
  # Iterate through the pixels and determine the color (black/white or custom color gradients)
  for x in range(image.width):
      for y in range(image.height):
          pixel = gray_image.getpixel((x, y))
          # Calculate the segment size
          segment_size = 256 // num_segments
          if pixel < (segment_size * 1.5):
              black_intensity = (segment_size - pixel) * (255 // segment_size)
              black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))  # Black-to-White
          elif pixel < 2 * segment_size:
              # Calculate the intensity of the custom color based on the pixel value
              custom_intensity = (pixel - segment_size) * (255 // segment_size)
              custom_color_image.putpixel((x, y), custom_color)  # Custom Color
  # Apply Floyd-Steinberg dithering to reduce black/white and custom color gradients to 2 colors
  black_white_dithered = black_white_image.convert('1', dither=Image.FLOYDSTEINBERG)
  custom_color_dithered = custom_color_image.convert('1', dither=Image.FLOYDSTEINBERG)
  custom_color_data = custom_color_dithered.load()
  img = Image.new('RGBA', black_white_dithered.size)
  return black_white_dithered, custom_color_dithered, img

# Function to convert a dithered image to a hexadecimal array
def dithered_image_to_byte_array(image, width, height):
    img_bytes = bytearray()
    CHUNKS = math.ceil(width / 8)
    for y in range(height):
        for chunk in range(CHUNKS):
            start = chunk * 8
            byte = 0
            for shift, x in enumerate(range(start, start+8)):
                p = image.getpixel((x, y)) if x < width else 1
                p = int(not p)
                byte |= p << (7-shift)
            img_bytes.append(byte)
    return img_bytes

def create_header_file(image, name):
    img_width, img_height = image.size
    img_bytes = dithered_image_to_byte_array(image, img_width, img_height)
    header_file = name + '.h'
    with open(header_file, 'w') as fp:
        #fp.write("static const uint8_t PROGMEM {}[] = ".format(name))
        #fp.write("{\n")
        row_count = 0
        for b in img_bytes:
            fp.write("0x{:02x}, ".format(b))
            row_count += 1
            if row_count >= 12:
                fp.write("\n")
                row_count = 0
        #fp.write("};\n")

black_white_dithered = None  # Initialize as global variables
custom_color_dithered = None  # Initialize as global variables

num_sets = 10
for iteration, _ in enumerate(range(num_sets)):
  iteration_str = 's'+str(iteration)
  generate_new_image(iteration)
  # Set dimensions for the images (adjust as needed)
  width = 50  # Replace with your desired width
  height = 50  # Replace with your desired height
  line_length = 12  # Number of pixels per line
  inverted_black_white_dithered = ImageOps.invert(black_white_dithered)
  inverted_custom_color_dithered = ImageOps.invert(custom_color_dithered)
  #display(image)
  #display(inverted_black_white_dithered)
  #display(inverted_custom_color_dithered)
  # Convert dithered images to hexadecimal arrays
  image_0_data = dithered_image_to_byte_array(black_white_dithered, width, height)
  image_1_data = dithered_image_to_byte_array(custom_color_dithered, width, height)

  create_header_file(black_white_dithered, iteration_str+'_0')
  create_header_file(custom_color_dithered, iteration_str+'_1')

  with open(iteration_str+'_0.h', 'r') as file_0, open(iteration_str+'_1.h', 'r') as file_1:
      image_0_content = file_0.read()
      image_1_content = file_1.read()

  # Write the contents of image_0.h and image_1.h into image.h
  with open(iteration_str+'.h', 'w') as image_file:
      image_file.write(image_0_content)
      image_file.write(image_1_content)

import zipfile
file_paths = []
for iteration, _ in enumerate(range(num_sets)):
  iteration_str = 's'+str(iteration)
  file_paths.append(iteration_str+'_0.h')
  file_paths.append(iteration_str+'_1.h')
zip_file_name = "header_files.zip"
with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for file in file_paths:
        zipf.write(file)

# Download the ZIP file
from google.colab import files
files.download(zip_file_name)


In [ ]:
from PIL import Image, ImageOps, ImageDraw
from IPython.display import display
import requests
from io import BytesIO
import zlib
import math
import argparse
from google.colab import files

uploaded = '/content/epd_image/1A14D231-330A-4880-9DC5-980A881E4EB0 (1).jpg'
img = Image.open(uploaded)
image = img.resize((200, 200), Image.ANTIALIAS)
gray_image = ImageOps.grayscale(image)
num_segments = 3
# Create empty images for black/white and custom color gradients
black_white_image = Image.new('RGB', image.size)
custom_color_image = Image.new('RGB', image.size)
# Define your custom color (e.g., green as (0, 255, 0))
custom_color = (255, 0, 0)
# Iterate through the pixels and determine the color (black/white or custom color gradients)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))
        # Calculate the segment size
        segment_size = 256 // num_segments
        if pixel < segment_size:
            black_intensity = (segment_size - pixel) * (255 // segment_size)
            black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))  # Black-to-White
        elif pixel < 2 * segment_size:
            # Calculate the intensity of the custom color based on the pixel value
            custom_intensity = (pixel - segment_size) * (255 // segment_size)
            custom_color_image.putpixel((x, y), custom_color)  # Custom Color
# Apply Floyd-Steinberg dithering to reduce black/white and custom color gradients to 2 colors
black_white_dithered = black_white_image.convert('1', dither=Image.FLOYDSTEINBERG)
custom_color_dithered = custom_color_image.convert('1', dither=Image.FLOYDSTEINBERG)
custom_color_data = custom_color_dithered.load()
img = Image.new('RGBA', black_white_dithered.size)

# Function to convert a dithered image to a hexadecimal array
def dithered_image_to_byte_array(image, width, height):
    img_bytes = bytearray()
    CHUNKS = math.ceil(width / 8)
    for y in range(height):
        for chunk in range(CHUNKS):
            start = chunk * 8
            byte = 0
            for shift, x in enumerate(range(start, start+8)):
                p = image.getpixel((x, y)) if x < width else 1
                p = int(not p)
                byte |= p << (7-shift)
            img_bytes.append(byte)
    return img_bytes

def create_header_file(image, name):
    img_width, img_height = image.size
    img_bytes = dithered_image_to_byte_array(image, img_width, img_height)
    header_file = name + '.h'
    with open(header_file, 'w') as fp:
        fp.write("static const uint8_t PROGMEM {}[] = ".format(name))
        fp.write("{\n")
        row_count = 0
        for b in img_bytes:
            fp.write("0x{:02x}, ".format(b))
            row_count += 1
            if row_count >= 12:
                fp.write("\n")
                row_count = 0
        fp.write("};\n")

width = 200  # Replace with your desired width
height = 200  # Replace with your desired height
line_length = 12  # Number of pixels per line
inverted_black_white_dithered = ImageOps.invert(black_white_dithered)
inverted_custom_color_dithered = ImageOps.invert(custom_color_dithered)
display(image)
display(inverted_black_white_dithered)
display(inverted_custom_color_dithered)
#Convert dithered images to hexadecimal arrays
image_0_data = dithered_image_to_byte_array(black_white_dithered, width, height)
image_1_data = dithered_image_to_byte_array(custom_color_dithered, width, height)
create_header_file(black_white_dithered, 's_0')
create_header_file(custom_color_dithered, 's_1')

with open('s_0.h', 'r') as file_0, open('s_1.h', 'r') as file_1:
    image_0_content = file_0.read()
    image_1_content = file_1.read()

# Write the contents of image_0.h and image_1.h into image.h
with open('s.h', 'w') as image_file:
    image_file.write(image_0_content)
    image_file.write(image_1_content)


In [ ]:
display(image)

In [ ]:
%cd /content/
import requests

url = 'https://raw.githubusercontent.com/adafruit/Adafruit-Thermal-Printer-Library/master/python/image_to_file.py'
response = requests.get(url)

if response.status_code == 200:
    with open('image_to_file.py', 'wb') as file:
        file.write(response.content)
    print('File downloaded successfully.')
else:
    print('Failed to download the file.')

In [ ]:
!python image_to_file.py -d BWR 'image.png'


In [ ]:
import shutil
import os

!pip install Pillow
!git clone https://github.com/bitbank2/epd_image.git
import os
os.chdir('/content/epd_image')
!make
response = requests.get('https://source.unsplash.com/random/200x200')
image = Image.open(BytesIO(response.content))
image.save('/content/epd_image/image.bmp')  # Adjust quality as needed

!./epd_image --BWR --DITHER 'image.bmp' 'image.h'

In [ ]:
os.chdir('/content/epd_image')
response = requests.get('https://source.unsplash.com/random/200x200')
image = Image.open(BytesIO(response.content))
image.save('/content/epd_image/image.jpg', quality=90)
!./epd_image --BWR --DITHER 'image.jpg' 'image.h'
display(image)

In [ ]:
!git clone https://github.com/zkwip/E-Paper-Image-Creator.git

In [ ]:
%cd E-Paper-Image-Creator


In [ ]:

upscale_factor = 8

# Create a new image with upscaled dimensions
upscaled_img = Image.new('RGBA', (combined_output.width * upscale_factor, combined_output.height * upscale_factor))

# Iterate through the original image and copy each pixel to the upscaled image
for x in range(combined_output.width):
    for y in range(combined_output.height):
        pixel = combined_output.getpixel((x, y))
        for i in range(upscale_factor):
            for j in range(upscale_factor):
                upscaled_img.putpixel((x * upscale_factor + i, y * upscale_factor + j), pixel)

# Save the upscaled image as a PDF file
upscaled_img = upscaled_img.convert('RGB')
upscaled_img.save('upscaled.pdf', 'PDF')
display(upscaled_img)

In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

response = requests.get('https://source.unsplash.com/random/2000x2000')
image = Image.open(BytesIO(response.content))
gray_image = ImageOps.grayscale(image)
custom_color = (255, 0, 0)
for x in range(image.width):
    for y in range(image.height):
        pixel = gray_image.getpixel((x, y))
        segment_size = 256 // 3
        if pixel < segment_size:
            black_intensity = (segment_size - pixel) * (255 // segment_size)
            black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))
        elif pixel < 2 * segment_size:
            custom_intensity = (pixel - segment_size) * (255 // segment_size)
            custom_color_image.putpixel((x, y), custom_color)
black_white_dithered = black_white_image.convert('1', dither=Image.FLOYDSTEINBERG)
custom_color_dithered = custom_color_image.convert('1', dither=Image.FLOYDSTEINBERG)

transparent_image = Image.new('RGBA', custom_color_dithered.size, (0, 0, 0, 0))

custom_color_data = custom_color_dithered.load()
transparent_data = transparent_image.load()
width, height = custom_color_dithered.size

img = Image.new('RGBA', black_white_dithered.size)
for x in range(width):
    for y in range(height):
        if custom_color_data[x, y] == 0:  # Check for black pixels
            img.putpixel((x, y), (0, 0, 0, 0))
        else:
            img.putpixel((x, y), custom_color + (255,))

combined_output = Image.new('RGB', (image.width, image.height))
combined_output.paste(black_white_dithered, (0, 0))
combined_output.paste(img, (0, 0), mask=img)

display(combined_output)

In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

def process_image_with_colors(image_url, custom_colors):
  response = requests.get(image_url)
  image = Image.open(BytesIO(response.content))
  gray_image = ImageOps.grayscale(image)

  black_white_image = Image.new('RGB', image.size)

  for i, color in enumerate(custom_colors):
    globals()[f'custom_color_image{i}'] = Image.new('RGB', image.size)

  for x in range(image.width):
    for y in range(image.height):
      pixel = gray_image.getpixel((x, y))
      segment_size = 256 // len(custom_colors)
      if pixel < segment_size:
        black_intensity = (segment_size - pixel) * (255 // segment_size)
        black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))
      elif pixel < 2 * segment_size:
        custom_intensity = (pixel - segment_size) * (255 // segment_size)
        globals()[f'custom_color_image{i}'].putpixel((x, y), custom_color)
  black_white_dithered = black_white_image.convert('1', dither=Image.FLOYDSTEINBERG)
  combined_output = Image.new('RGB', (image.width, image.height))
  combined_output.paste(black_white_dithered, (0, 0))
  for i, color in enumerate(custom_colors):
    cc_dithered = globals()[f'custom_color_image{i}'].convert('1', dither=Image.FLOYDSTEINBERG)
    cc_transparent = Image.new('RGBA', cc_dithered.size, (0, 0, 0, 0))
    cc_data = cc_dithered.load()
    cc_transparent_data = cc_transparent.load()
    width, height = cc_dithered.size
    img = Image.new('RGBA', black_white_dithered.size)
    for x in range(width):
      for y in range(height):
        if custom_color_data[x, y] == 0:  # Check for black pixels
          img.putpixel((x, y), (0, 0, 0, 0))
        else:
          img.putpixel((x, y), custom_color + (255,))
    combined_output.paste(img, (0, 0), mask=img)

custom_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # Define your custom colors
image_url = 'https://source.unsplash.com/random/200x200'  # URL of the image
process_image_with_colors(image_url, custom_colors)
display(combined_output)

In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO

def process_image_with_colors(image_url, custom_colors):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    gray_image = ImageOps.grayscale(image)

    black_white_image = Image.new('RGB', image.size)

    for i, color in enumerate(custom_colors):
        globals()[f'custom_color_image{i}'] = Image.new('RGB', image.size)

    for x in range(image.width):
      for y in range(image.height):
        pixel = gray_image.getpixel((x, y))
        segment_size = 256 // len(custom_colors)
        color_index = pixel // segment_size
        if color_index >= len(custom_colors):
          color_index = len(custom_colors) - 1
          black_intensity = (segment_size - (pixel % segment_size)) * (255 // segment_size)
          black_white_image.putpixel((x, y), (black_intensity, black_intensity, black_intensity))
          globals()[f'custom_color_image{color_index}'].putpixel((x, y), custom_colors[color_index])

    black_white_dithered = black_white_image.convert('1', dither=Image.FLOYDSTEINBERG)
    combined_output = Image.new('RGB', (image.width, image.height))
    combined_output.paste(black_white_dithered, (0, 0))

    for i, color in enumerate(custom_colors):
        cc_dithered = globals()[f'custom_color_image{i}'].convert('1', dither=Image.FLOYDSTEINBERG)
        cc_transparent = Image.new('RGBA', cc_dithered.size, (0, 0, 0, 0))
        cc_data = cc_dithered.load()
        cc_transparent_data = cc_transparent.load()
        width, height = cc_dithered.size
        img = Image.new('RGBA', black_white_dithered.size)
        for x in range(width):
            for y in range(height):
                if cc_data[x, y] == 0:  # Check for black pixels
                    img.putpixel((x, y), (0, 0, 0, 0))
                else:
                    img.putpixel((x, y), color + (255,))
        combined_output.paste(img, (0, 0), mask=img)
        display(img)

    display(combined_output)

# Example usage:
custom_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # Define your custom colors
image_url = 'https://source.unsplash.com/random/200x200'  # URL of the image
process_image_with_colors(image_url, custom_colors)


In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO
import random


def process_image_with_colors(image_url, custom_colors):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    gray_image = ImageOps.grayscale(image)

    display(image)
    #display(gray_image)

    combined_output = Image.new('RGB', (image.width, image.height))

    for i, color in enumerate(custom_colors):
        color_image = Image.new('RGB', image.size)
        for x in range(image.width):
            for y in range(image.height):
                pixel = gray_image.getpixel((x, y))
                segment_size = 256 // len(custom_colors)
                color_index = pixel // segment_size
                if color_index >= len(custom_colors):
                    color_index = len(custom_colors) - 3
                if color_index == i:
                    color_image.putpixel((x, y), custom_colors[i])

        cc_dithered = color_image.convert('1', dither=Image.FLOYDSTEINBERG)
        cc_transparent = Image.new('RGBA', cc_dithered.size, (0, 0, 0, 0))
        cc_data = cc_dithered.load()
        cc_transparent_data = cc_transparent.load()
        width, height = cc_dithered.size
        img = Image.new('RGBA', image.size)
        for x in range(width):
            for y in range(height):
                if cc_data[x, y] == 0:  # Check for black pixels
                    img.putpixel((x, y), (0, 0, 0, 0))
                else:
                    img.putpixel((x, y), color + (255,))
        #display(img)
        combined_output.paste(img, (0, 0), mask=img)

    display(combined_output)
    combined_output.save('combined_output.png')

number_of_colors = random.randint(2, 10)
custom_colors = []

for _ in range(number_of_colors):
    custom_colors.append((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))
image_url = 'https://source.unsplash.com/random/720x720'  # URL of the image
process_image_with_colors(image_url, custom_colors)


In [ ]:
import cv2
import numpy as np
import random
from PIL import Image
import requests
from io import BytesIO

# Load your image from a URL
image = Image.open('combined_output.png')

# Get the image dimensions
width, height = image.size

# Convert the PIL Image to a NumPy array
image_np = np.array(image)

# Generate random destination points for the maximum warping state
# Adjust the range to control the extent of warping
dst_points_max_warp = np.array([
    [random.randint(0, width), random.randint(0, height)],
    [random.randint(0, width), random.randint(0, height)],
    [random.randint(0, width), random.randint(0, height)],
    [random.randint(0, width), random.randint(0, height)]
], dtype=np.float32)

# Create an animation from normal to max warping and back to normal
num_frames = 300  # Number of frames in the animation (30 each way)
output_filename = 'morphing_animation.mp4'

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec
out = cv2.VideoWriter(output_filename, fourcc, 60.0, (width, height))  # Set the frame rate to 60fps

for i in range(num_frames * 2):
    t = i / (num_frames * 2 - 1)  # Progress from 0 to 1 and back to 0
    if t <= 0.5:
        # Morph from normal to max warping
        src_points = np.array([
            [0, 0],
            [width, 0],
            [0, height],
            [width, height]
        ], dtype=np.float32)
        src_points_intermediate = (2 * t) * src_points + (1 - 2 * t) * dst_points_max_warp
    else:
        # Morph from max warping back to normal
        src_points_intermediate = (2 - 2 * t) * dst_points_max_warp + (2 * t - 1) * src_points

    # Calculate the perspective transformation matrix
    perspective_matrix = cv2.getPerspectiveTransform(src_points_intermediate, src_points)  # Use src_points here

    # Warp the image
    warped_image = cv2.warpPerspective(image_np, perspective_matrix, (width, height))

    # Write the frame to the video
    out.write(warped_image)

out.release()



In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO
import random
import cv2
import numpy as np

def process_image_with_colors(image_url, custom_colors):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    gray_image = ImageOps.grayscale(image)

    display(image)
    #display(gray_image)

    combined_output = Image.new('RGB', (image.width, image.height))

    for i, color in enumerate(custom_colors):
        color_image = Image.new('RGB', image.size)
        for x in range(image.width):
            for y in range(image.height):
                pixel = gray_image.getpixel((x, y))
                segment_size = 256 // len(custom_colors)
                color_index = pixel // segment_size
                if color_index >= len(custom_colors):
                    color_index = len(custom_colors) - 3
                if color_index == i:
                    color_image.putpixel((x, y), custom_colors[i])

        cc_dithered = color_image.convert('1', dither=Image.FLOYDSTEINBERG)
        cc_transparent = Image.new('RGBA', cc_dithered.size, (0, 0, 0, 0))
        cc_data = cc_dithered.load()
        cc_transparent_data = cc_transparent.load()
        width, height = cc_dithered.size
        img = Image.new('RGBA', image.size)
        for x in range(width):
            for y in range(height):
                if cc_data[x, y] == 0:  # Check for black pixels
                    img.putpixel((x, y), (0, 0, 0, 0))
                else:
                    img.putpixel((x, y), color + (255,))
        #display(img)
        combined_output.paste(img, (0, 0), mask=img)

    display(combined_output)
    combined_output.save('combined_output.png')


pure_colors = [
    (255, 0, 0),     # Red
    (0, 0, 255),     # Blue
    (0, 255, 0),     # Green
    (255, 255, 0),   # Yellow
    (0, 0, 0),       # Black
    (255, 255, 255), # White
    (128, 128, 128), # Grey
    (255, 165, 0),   # Orange
    (128, 0, 128)    # Purple
]

number_of_colors = random.randint(2, 4)
custom_colors = []
custom_colors = random.sample(pure_colors, number_of_colors)


image_url = 'https://source.unsplash.com/random/720x720'  # URL of the image
process_image_with_colors(image_url, custom_colors)

# Load your image from a URL
image = Image.open('combined_output.png')

# Get the image dimensions
width, height = image.size

# Convert the PIL Image to a NumPy array
image_np = np.array(image)



# Generate random destination points for the maximum warping state
# Adjust the range to control the extent of warping
dst_points_max_warp = np.array([
    [random.randint(0, width), random.randint(0, height)],
    [random.randint(0, width), random.randint(0, height)],
    [random.randint(0, width), random.randint(0, height)],
    [random.randint(0, width), random.randint(0, height)]
], dtype=np.float32)

# Create an animation from normal to max warping and back to normal
num_frames = 1000  # Number of frames in the animation (30 each way)
output_filename = 'morphing_animation.mp4'

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec
out = cv2.VideoWriter(output_filename, fourcc, 60.0, (width, height))  # Set the frame rate to 60fps


for i in range(num_frames):
    if i > (num_frames / 2):
        break
    t = i / (num_frames - 1)  # Progress from 0 to 1
    src_points = np.array([
        [0, 0],
        [width, 0],
        [0, height],
        [width, height]
    ], dtype=np.float32)
    src_points_intermediate = t * dst_points_max_warp + (1 - t) * src_points

    # Calculate the perspective transformation matrix
    perspective_matrix = cv2.getPerspectiveTransform(src_points_intermediate, src_points)

    # Warp the image
    warped_image = cv2.warpPerspective(image_np, perspective_matrix, (width, height))

    # Write the frame to the video
    out.write(warped_image)

out.release()



In [ ]:
from PIL import Image, ImageOps
import requests
from io import BytesIO
import random
import cv2
import numpy as np

def process_image_with_colors(image_url, custom_colors):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    gray_image = ImageOps.grayscale(image)

    display(image)
    #display(gray_image)

    combined_output = Image.new('RGB', (image.width, image.height))

    for i, color in enumerate(custom_colors):
        color_image = Image.new('RGB', image.size)
        for x in range(image.width):
            for y in range(image.height):
                pixel = gray_image.getpixel((x, y))
                segment_size = 256 // len(custom_colors)
                color_index = pixel // segment_size
                if color_index >= len(custom_colors):
                    color_index = len(custom_colors) - 3
                if color_index == i:
                    color_image.putpixel((x, y), custom_colors[i])

        cc_dithered = color_image.convert('1', dither=Image.FLOYDSTEINBERG)
        cc_transparent = Image.new('RGBA', cc_dithered.size, (0, 0, 0, 0))
        cc_data = cc_dithered.load()
        cc_transparent_data = cc_transparent.load()
        width, height = cc_dithered.size
        img = Image.new('RGBA', image.size)
        for x in range(width):
            for y in range(height):
                if cc_data[x, y] == 0:  # Check for black pixels
                    img.putpixel((x, y), (0, 0, 0, 0))
                else:
                    img.putpixel((x, y), color + (255,))
        #display(img)
        combined_output.paste(img, (0, 0), mask=img)

    display(combined_output)
    #combined_output.save('combined_output.png')

# Generate the animation frames and save as a video
num_frames = 250  # Number of frames in the animation (30 each way)
output_filename = 'morphing_animation.mp4'

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Video codec
out = cv2.VideoWriter(output_filename, fourcc, 60.0, (width, height))  # Set the frame rate to 60fps

for i in range(num_frames * 2):
    t = i / (num_frames * 2 - 1)  # Progress from 0 to 1 and back to 0
    if t <= 0.5:
        # Morph from normal to max warping
        src_points = np.array([
            [0, 0],
            [width, 0],
            [0, height],
            [width, height]
        ], dtype=np.float32)
        src_points_intermediate = (2 * t) * src_points + (1 - 2 * t) * dst_points_max_warp
    else:
        # Morph from max warping back to normal
        src_points_intermediate = (2 - 2 * t) * dst_points_max_warp + (2 * t - 1) * src_points

    # Calculate the perspective transformation matrix
    perspective_matrix = cv2.getPerspectiveTransform(src_points_intermediate, src_points)  # Use src_points here

    # Warp the image
    warped_image = cv2.warpPerspective(image_np, perspective_matrix, (width, height))

    # Convert the NumPy array back to a PIL Image
    warped_pil_image = Image.fromarray(warped_image)

    # Save the frame as a video frame
    frame_path = 'frame_{:04d}.png'.format(i)
    warped_pil_image.save(frame_path)
    out.write(warped_image)

out.release()

# Combine frames into a video using FFmpeg
import subprocess

# Adjust the command to match your system's FFmpeg installation
ffmpeg_cmd = [
    'ffmpeg',
    '-framerate', '60',  # Frame rate (adjust as needed)
    '-i', 'frame_%04d.png',  # Input frame pattern
    '-c:v', 'libx264',
    '-pix_fmt', 'yuv420p',
    '-y',  # Overwrite existing output file if it exists
    'final_morphing_animation.mp4'  # Output video file name
]

subprocess.run(ffmpeg_cmd)


In [ ]:
import requests
from IPython.display import HTML
import time

# Giphy API defaults
giphy = {
    "baseURL": "https://api.giphy.com/v1/gifs/",
    "apiKey": "0UTRbFtkMxAplrohufYco5IY74U8hOes",
    "tag": "fail",
    "type": "random",
    "rating": "pg-13"
}

# Giphy API URL
giphyURL = f"{giphy['baseURL']}{giphy['type']}?api_key={giphy['apiKey']}&tag={giphy['tag']}&rating={giphy['rating']}"

# Function to get a random GIF
def get_random_gif():
    response = requests.get(giphyURL)
    if response.status_code == 200:
        gif_data = response.json()
        return gif_data["data"]["images"]["original"]["url"]
    else:
        return None

# Function to display the GIF in Colab
def display_gif(url):
    return HTML(f'<img src="{url}" alt="GIF">')

# Display a random GIF
gif_url = get_random_gif()
display_gif(gif_url)


In [ ]:
import imageio
import numpy as np
from PIL import Image, ImageOps
import requests
from io import BytesIO
import random
import subprocess
from PIL import ImageSequence

def save_frames_as_png(gif_url):
    response = requests.get(gif_url)
    gif = Image.open(BytesIO(response.content))

    frame_filenames = []
    for i, frame in enumerate(ImageSequence.Iterator(gif)):
        frame_filename = f'frame_{i}.png'
        frame.save(frame_filename)
        frame_filenames.append(frame_filename)

    return frame_filenames

def process_frame_with_colors(frame, custom_colors):
    gray_frame = ImageOps.grayscale(frame)
    processed_frame = Image.new('RGBA', frame.size)

    for i, color in enumerate(custom_colors):
        for x in range(frame.width):
            for y in range(frame.height):
                pixel = gray_frame.getpixel((x, y))
                segment_size = 256 // len(custom_colors)
                color_index = pixel // segment_size
                if color_index >= len(custom_colors):
                    color_index = len(custom_colors) - 3
                if color_index == i:
                    processed_frame.putpixel((x, y), custom_colors[i])

    return processed_frame


pure_colors = [
    (255, 0, 0),     # Red
    (0, 0, 255),     # Blue
    (0, 255, 0),     # Green
    (255, 255, 0),   # Yellow
    (0, 0, 0),       # Black
    (255, 255, 255), # White
    (128, 128, 128), # Grey
    (255, 165, 0),   # Orange
    (128, 0, 128)    # Purple
]


gif_url = get_random_gif()  # Use your method to get a random GIF URL
frame_filenames = save_frames_as_png(gif_url)

# Choose a random number of colors (x) from the list
number_of_colors = random.randint(2, 4)
custom_colors = random.sample(pure_colors, number_of_colors)

processed_frame_filenames = []

for i, frame_filename in enumerate(frame_filenames):
    frame = Image.open(frame_filename)
    processed_frame = process_frame_with_colors(frame, custom_colors)
    processed_frame_filename = f'processed_frame_{i}.png'
    processed_frame.save(processed_frame_filename)
    processed_frame_filenames.append(processed_frame_filename)

# Calculate the desired frame rate and number of times to repeat frames
frame_rate = len(processed_frame_filenames) / 8  # 8 seconds total duration
repeats = int(8 / len(processed_frame_filenames))

# Resize frames to 720x720
processed_frames_resized = []
for frame_filename in processed_frame_filenames:
    frame = Image.open(frame_filename)
    frame = frame.resize((720, 720), Image.ANTIALIAS)
    for _ in range(repeats):
        processed_frames_resized.append(frame)

def create_video_from_frames(frame_filenames, output_filename, frame_rate):
    images = [Image.open(frame_filename) for frame_filename in frame_filenames]

    with imageio.get_writer(output_filename, mode='I', fps=frame_rate) as writer:
        for image in images:
            image_array = np.array(image)
            writer.append_data(image_array)

# Usage:
create_video_from_frames(processed_frames_resized, 'output_video.mp4', frame_rate)

In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO
import random
from PIL import ImageSequence

def process_gif_with_colors(gif_filename, custom_colors):
    gif = Image.open(gif_filename)

    if gif.is_animated:
        gif_frames = []

        for frame in ImageSequence.Iterator(gif):
            gray_frame = ImageOps.grayscale(frame)
            color_frames = []

            for i, color in enumerate(custom_colors):
                color_frame = Image.new('RGB', frame.size, color)
                for x in range(frame.width):
                    for y in range(frame.height):
                        pixel = gray_frame.getpixel((x, y))
                        segment_size = 256 // len(custom_colors)
                        color_index = pixel // segment_size
                        if color_index == i:
                            color_frame.putpixel((x, y), color)

                cc_dithered = color_frame.convert('1', dither=Image.FLOYDSTEINBERG)
                cc_transparent = Image.new('RGBA', cc_dithered.size, (0, 0, 0, 0))
                cc_data = cc_dithered.load()
                cc_transparent_data = cc_transparent.load()
                width, height = cc_dithered.size
                img = Image.new('RGBA', frame.size)
                for x in range(width):
                    for y in range(height):
                        if cc_data[x, y] == 0:  # Check for black pixels
                            img.putpixel((x, y), (0, 0, 0, 0))
                        else:
                            img.putpixel((x, y), color + (255,))
                color_frames.append(img)

            gif_frames.extend(color_frames)

        gif_with_colors = Image.new('RGBA', gif.size)
        gif_with_colors.save('gif_with_colors.gif', save_all=True, append_images=gif_frames)
    else:
        print("The provided image is not an animated GIF.")

# Predefined list of colors
pure_colors = [
    (255, 0, 0),     # Red
    (0, 0, 255),     # Blue
    (0, 255, 0),     # Green
    (255, 255, 0),   # Yellow
    (0, 0, 0),       # Black
    (255, 255, 255), # White
    (128, 128, 128), # Grey
    (255, 165, 0),   # Orange
    (128, 0, 128)    # Purple
]

# Choose a random number of colors (x) from the list
number_of_colors = random.randint(2, 4)
custom_colors = random.sample(pure_colors, number_of_colors)

gif_url = get_random_gif()  # Use the function from the previous response to get a random GIF URL
save_gif_locally(gif_url, 'downloaded.gif')
process_gif_with_colors('downloaded.gif', custom_colors)




In [ ]:
from PIL import Image, ImageOps
from IPython.display import display
import requests
from io import BytesIO
from ipywidgets import widgets

boolean_checkbox = False #@param {type: "boolean"}
my_list = ["a","b","c","d"]
checkboxes = []
for item in my_list:
    checkbox = widgets.Checkbox(description=item, value=True)
    checkboxes.append(checkbox)

widgets.VBox(checkboxes)

def brightness(color):
    # Calculate the brightness of a color (RGB)
    return 0.299 * color[0] + 0.587 * color[1] + 0.114 * color[2]

def process_image_with_colors(image_url, custom_colors, sort_by_brightness=True):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    gray_image = ImageOps.grayscale(image)

    display(gray_image)

    combined_output = Image.new('RGB', (image.width, image.height))

    if sort_by_brightness:
        custom_colors = sorted(custom_colors, key=brightness)

    for i, color in enumerate(custom_colors):
        color_image = Image.new('RGB', image.size)
        for x in range(image.width):
            for y in range(image.height):
                pixel = gray_image.getpixel((x, y))
                segment_size = 256 // len(custom_colors)
                color_index = pixel // segment_size
                if color_index >= len(custom_colors):
                    color_index = len(custom_colors) - 1
                if color_index == i:
                    color_image.putpixel((x, y), custom_colors[i])

        cc_dithered = color_image.convert('1', dither=Image.FLOYDSTEINBERG)
        cc_transparent = Image.new('RGBA', cc_dithered.size, (0, 0, 0, 0))
        cc_data = cc_dithered.load()
        cc_transparent_data = cc_transparent.load()
        width, height = cc_dithered.size
        img = Image.new('RGBA', image.size)
        for x in range(width):
            for y in range(height):
                if cc_data[x, y] == 0:  # Check for black pixels
                    img.putpixel((x, y), (0, 0, 0, 0))
                else:
                    img.putpixel((x, y), custom_colors[i] + (255,))
        combined_output.paste(img, (0, 0), mask=img)

    display(combined_output)
    display(image)

custom_colors = [
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (255, 255, 255),
    (255, 255, 0),
    (255, 0, 255),
    (0, 255, 255)
]
image_url = 'https://source.unsplash.com/random/800x800'  # URL of the image
process_image_with_colors(image_url, custom_colors, sort_by_brightness=True)
